In [3]:
#Importing all the require packages
import torch
from torch import nn
import torch.nn.functional as F
from torchvision.datasets import Imagenette
import torchvision.transforms as transforms
import torchmetrics
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

In [6]:
#  Data augmentation for regularization
# Preparing data
train_transforms_reg_data_aug = transforms.Compose([
    transforms.RandomResizedCrop(64),  # Change crop size to 64x64
    transforms.RandomHorizontalFlip(),  # Flip horizontally randomly
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2),  # Adjust color brightness, contrast, saturation, and hue
    transforms.RandomRotation(15),  # Rotate randomly up to 15 degrees
    transforms.ToTensor(),  # Convert PIL image to tensor
    transforms.Normalize((0.4912, 0.4821, 0.4455), (0.2370, 0.2437, 0.2626)),  # Normalize image tensor
])



test_transforms = transforms.Compose([
    transforms.CenterCrop(160),
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize((0.4912, 0.4821, 0.4455), (0.2370, 0.2437, 0.2626)),
])


train_dataset = Imagenette("data/imagenette/train/", split="train", size="160px", download=False, transform=train_transforms_reg_data_aug)
num_classes = len(train_dataset.classes)

train_set_size = int(len(train_dataset) * 0.9)
val_set_size = len(train_dataset) - train_set_size
seed = torch.Generator().manual_seed(42)
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_set_size, val_set_size], generator=seed)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, num_workers=8, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=128, num_workers=8, shuffle=False)

test_dataset = Imagenette("data/imagenette/test/", split="val", size="160px", download=False, transform=test_transforms)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, num_workers=8, shuffle=False)



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [7]:
#Initailizing/defining AllConvNet model
class AllConvNet(pl.LightningModule):
    def __init__(self, num_classes=num_classes):
        super().__init__()

        # Define convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)

        # Pooling layers
        self.pool = nn.MaxPool2d(2, stride=2)

        # Output convolutional layer
        self.conv4 = nn.Conv2d(128, num_classes, kernel_size=3, padding=1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)

        x = F.relu(self.conv3(x))
        x = self.pool(x)

        # Final convolutional layer with kernel size equal to the size of the feature map
        x = F.relu(self.conv4(x))

        # Global average pooling
        x = F.avg_pool2d(x, x.shape[2:])

        return x.view(x.size(0), -1)

    def training_step(self, batch, batch_idx):
      x, y = batch
      y_hat = self(x)
      loss = F.cross_entropy(y_hat, y)
      self.log("train_loss", loss)

      # Calculate accuracy
      pred_labels = torch.argmax(y_hat, dim=1)
      accuracy = torchmetrics.functional.accuracy(pred_labels, y, task="multiclass", num_classes=num_classes)
      self.log("train_accuracy", accuracy, on_step=False, on_epoch=True)

      return loss

    # validation_step method
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        pred_labels = torch.argmax(y_hat, dim=1)
        accuracy = torchmetrics.functional.accuracy(pred_labels, y, task="multiclass", num_classes=num_classes)
        self.log("val_loss", loss)
        self.log("val_accuracy", accuracy, on_step=False, on_epoch=True)

    # test_step method
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        pred_labels = torch.argmax(y_hat, dim=1)
        accuracy = torchmetrics.functional.accuracy(pred_labels, y, task="multiclass", num_classes=num_classes)
        self.log("test_loss", loss)
        self.log("test_accuracy", accuracy, on_step=False, on_epoch=True)




    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer



In [9]:
# Model instantiation
model = AllConvNet()

# Early stopping callback
early_stop_callback = EarlyStopping(monitor="val_loss", mode="min", patience=5)

# Model checkpoint callback
checkpoint_callback = ModelCheckpoint(monitor="val_loss", mode="min")

# Fit the model
trainer = pl.Trainer(callbacks=[early_stop_callback, checkpoint_callback])
trainer.fit(model, train_loader, val_loader)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/utilities.py:73: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type      | Params
------------------------------------
0 | conv1 | Conv2d    | 896   
1 | conv2 | Conv2d    | 18.5 K
2 | conv3 | Conv2d    | 73.9 K
3 | pool  | MaxPool2d | 0     
4 | conv4 | Conv2d    | 11.5 K
------------------------------------
104 K     Trainable params
0         Non-trainable params
104 K     Total params
0.419     Total 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [10]:
Train_metrics = trainer.logged_metrics
# Evaluation
print(Train_metrics)
print("Train Metrics")
print("Train Accuracy:", Train_metrics["train_accuracy"])
print("Train Loss:", Train_metrics["train_loss"])


Train Metrics
Train Accuracy: tensor(0.6081)
Train Loss: tensor(1.2220)


In [11]:
trainer.test(model, test_loader)
Test_metrics = trainer.logged_metrics
# Evaluation
print("Test Metrics")
print("Test Accuracy:", Test_metrics["test_accuracy"])
print("Test Loss:", Test_metrics["test_loss"])

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.6792356967926025     │
│         test_loss         │    1.1280313730239868     │
└───────────────────────────┴───────────────────────────┘

Test Metrics
Test Accuracy: tensor(0.6792)
Test Loss: tensor(1.1280)


In [12]:
#Saving the All_ConvN model
torch.save(model.state_dict(), 'Regularization_Model_data_aug.pth')